# Qiskit Coding Exercise: BB84 Quantum Key Distribution

This notebook guides you through simulating the BB84 protocol to securely distribute a cryptographic key. We will simulate Alice, Bob, and an eavesdropper (Eve) to see how quantum mechanics guarantees security.

**Setup:**
You will need `qiskit` (v1.0+), `qiskit-aer` and `numpy`.
`pip install qiskit qiskit-aer numpy`

### Introduction: Quantum Key Distribution (QKD)

**Objective:** Apply the principles of superposition and measurement to cryptography.

**The Problem:** How can Alice and Bob agree on a secret random key if they only have a public channel that Eve might be listening to?

**The Quantum Solution (BB84):**
In classical physics, eavesdropping is passive (copying a file doesn't change it). In quantum physics, **observation causes disturbance**.
* If Eve tries to read the quantum bits sent by Alice, she will inevitably change their state.
* Alice and Bob can detect this disturbance (errors) in their data.
* If the error rate is low, they know no one listened.

This protocol was proposed by Bennett and Brassard in 1984 (hence BB84).

In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

--- 

## Phase 1: Engage (The Setup)

### 1. Generating Randomness
Alice needs two random strings of bits:
1.  **Data Bits:** The actual key material (0s and 1s).
2.  **Basis Bits:** Which basis to use for encoding (Z-basis or X-basis).

**The 4 BB84 States:**
Alice encodes her bits into one of 4 possible states based on her choice of basis:

* **Z-Basis (Standard):** $|0\rangle$ (Bit 0), $|1\rangle$ (Bit 1)
* **X-Basis (Hadamard):** $|+\rangle$ (Bit 0), $|-\rangle$ (Bit 1)

$$ |+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}, \quad |-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}} $$

This ensures that if Bob measures in the *wrong* basis (e.g., Alice sends $|0\rangle$ but Bob measures in X), he gets a completely random result (50% 0, 50% 1).

In [4]:
n = 20 # Number of qubits to send

# Set a seed for reproducibility
np.random.seed(42)

# 1. Alice generates random bits (The Key Material)
alice_bits = np.random.randint(2, size=n)

# 2. Alice generates random bases (0 = Z-basis, 1 = X-basis)
alice_bases_int = np.random.randint(2, size=n)
alice_bases = np.array(['Z' if b == 0 else 'X' for b in alice_bases_int])

print("Alice's Bits: ", alice_bits)
print("Alice's Bases:", alice_bases)

Alice's Bits:  [0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0]
Alice's Bases: ['X' 'Z' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'Z' 'Z' 'X' 'X' 'X' 'Z' 'X' 'Z'
 'Z' 'Z']


### 2. Encoding the Qubits
Now we write a function to create the quantum circuits corresponding to Alice's choices.

**Logic:**
* If Bit=1: Apply X gate (flip $|0\rangle$ to $|1\rangle$).
* If Basis=X: Apply H gate (rotate Z basis to X basis).

In [5]:
def encode_qubits(bits, bases):
    circuits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        
        # Encode Bit
        if bit == 1:
            qc.x(0)
            
        # Encode Basis
        if basis == 'X':
            qc.h(0)
            
        circuits.append(qc)
    return circuits

alice_circuits = encode_qubits(alice_bits, alice_bases)

## Phase 2: Transmission & Measurement

Alice sends these qubits to Bob over a fiber optic cable. Bob, not knowing Alice's basis choice, picks his own random basis to measure each qubit.

**Key Concept: Sifting**
Because Bob guesses the basis randomly, he will guess correctly only 50% of the time.
* **Basis Match:** Alice sent $|0\rangle$ (Z), Bob measured in Z. Result: 0 (Deterministic, Perfect correlation).
* **Basis Mismatch:** Alice sent $|0\rangle$ (Z), Bob measured in X. Result: 0 or 1 (Random, No correlation).

Later, they will publicly reveal *only which bases they used* (not the bits!) and discard the instances where they mismatched.

In [6]:
# Bob chooses random bases
bob_bases_int = np.random.randint(2, size=n)
bob_bases = np.array(['Z' if b == 0 else 'X' for b in bob_bases_int])

print("Bob's Bases:  ", bob_bases)

Bob's Bases:   ['Z' 'Z' 'X' 'X' 'X' 'X' 'X' 'Z' 'X' 'X' 'Z' 'X' 'Z' 'X' 'Z' 'X' 'X' 'Z'
 'Z' 'Z']


### 3. Bob Measures
Bob receives the qubits and measures them.
**Logic:**
* If Bob chooses Z-basis: Measure directly.
* If Bob chooses X-basis: Apply H gate first (to rotate X back to Z), then measure.

In [7]:
def measure_qubits(circuits, bases):
    measurements = []
    simulator = AerSimulator()
    
    for qc, basis in zip(circuits, bases):
        # We copy the circuit so we don't modify Alice's original list
        qc_bob = qc.copy()
        
        if basis == 'X':
            qc_bob.h(0)
            
        qc_bob.measure(0, 0)
        
        # Run Simulation
        result = simulator.run(qc_bob, shots=1, memory=True).result()
        bit = int(result.get_memory()[0])
        measurements.append(bit)
        
    return np.array(measurements)

bob_bits = measure_qubits(alice_circuits, bob_bases)
print("Bob's Results:", bob_bits)

Bob's Results: [1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1 0]


## Phase 3: Sifting (The Handshake)

Now Alice and Bob talk over a public channel (like the internet). They say: "Hey, for qubit 1 I used Z, for qubit 2 I used X..."

They **keep** the bits where their bases matched. They **discard** the rest.

**Task:** Implement the sifting logic.

In [8]:
match_indices = np.where(alice_bases == bob_bases)[0]

alice_key = alice_bits[match_indices]
bob_key = bob_bits[match_indices]

print("Bases Match at indices:", match_indices)
print("Alice's Sifted Key:", alice_key)
print("Bob's Sifted Key:  ", bob_key)
print(f"Key Length: {len(alice_key)} (approx 50% of {n})")

# Check if they match perfectly (They should in a noise-free simulation)
print("Do keys match?", np.array_equal(alice_key, bob_key))

Bases Match at indices: [ 1  2  3  4  5  6  8  9 10 13 16 17 18 19]
Alice's Sifted Key: [1 0 0 0 1 0 0 1 0 0 1 1 1 0]
Bob's Sifted Key:   [1 0 0 0 1 0 0 1 0 0 1 1 1 0]
Key Length: 14 (approx 50% of 20)
Do keys match? True


--- 

## Phase 4: The Attack (Eve)

Now let's introduce Eve. She performs an **Intercept-Resend Attack**.
1.  She intercepts the qubit from Alice.
2.  She measures it in a random basis (guessing, just like Bob).
3.  She prepares a *new* qubit based on her measurement result and sends it to Bob.

**The Problem for Eve:**
If she guesses the wrong basis, she collapses the state. For example:
* Alice sends $|0\rangle$ (Z).
* Eve measures in X. She gets $|+\rangle$ or $|-\rangle$ randomly.
* She sends $|+\rangle$ to Bob.
* Bob measures in Z (Matching Alice!). Ideally he should get $|0\rangle$ with 100% certainty. But because Eve sent $|+\rangle$, Bob now gets 0 or 1 with 50/50 probability.

This introduces **Errors** in the sifted key.

### 6. Simulating Eve
Eve is essentially a copy of Bob who sits in the middle. We will intercept the `alice_circuits`, measure them, and create `eve_circuits` to send to Bob.

In [9]:
def intercept_resend(circuits):
    eve_circuits = []
    eve_bases_int = np.random.randint(2, size=len(circuits))
    eve_bases = np.array(['Z' if b == 0 else 'X' for b in eve_bases_int])
    
    simulator = AerSimulator()
    
    for qc, basis in zip(circuits, eve_bases):
        # Eve measures
        qc_eve = qc.copy()
        if basis == 'X':
            qc_eve.h(0)
        qc_eve.measure(0, 0)
        
        result = simulator.run(qc_eve, shots=1, memory=True).result()
        bit = int(result.get_memory()[0])
        
        # Eve prepares new qubit for Bob
        new_qc = QuantumCircuit(1, 1)
        if bit == 1:
            new_qc.x(0)
        if basis == 'X':
            new_qc.h(0)
            
        eve_circuits.append(new_qc)
        
    return eve_circuits

# Run the attack
alice_circuits = encode_qubits(alice_bits, alice_bases) # Reset circuits
raw_key_attacked_circuits = intercept_resend(alice_circuits)

# Bob receives these attacked circuits
raw_key_attacked = measure_qubits(raw_key_attacked_circuits, bob_bases)

### 7. Checking for Errors (QBER)

After sifting, Alice and Bob compare a subset of their keys to estimate the **Quantum Bit Error Rate (QBER)**.

**Theoretical Prediction:**
* Eve guesses wrong basis 50% of the time.
* When she guesses wrong, she randomizes the state.
* Bob then has a 50% chance of measuring the wrong bit even if bases match.
* Total Error Rate $\approx 0.5 \times 0.5 = 25\%$.

**Task:** Calculate the error rate in the sifted key.

In [10]:
alice_key_part = alice_bits[match_indices]
bob_key_part = raw_key_attacked[match_indices]

errors = 0
for a, b in zip(alice_key_part, bob_key_part):
    if a != b:
        errors += 1
        
qber = errors / len(alice_key_part)

print("Alice Key:", alice_key_part)
print("Bob Key:  ", bob_key_part)
print(f"Errors: {errors} out of {len(alice_key_part)}")
print(f"QBER: {qber:.2f}")

if qber > 0.11:
    print("Security Check: EVE DETECTED! ABORT!")
else:
    print("Security Check: Safe. Proceed to Error Correction.")

Alice Key: [1 0 0 0 1 0 0 1 0 0 1 1 1 0]
Bob Key:   [1 1 0 1 1 0 0 1 0 0 1 1 1 0]
Errors: 2 out of 14
QBER: 0.14
Security Check: EVE DETECTED! ABORT!


### Note: Post-Processing Steps

Even if the QBER is low (below 11%), Alice and Bob are not done. They must perform two classical steps:

1.  **Error Correction (Information Reconciliation):** They communicate publicly to fix the few mismatched bits (using parity checks) without revealing the whole key.
2.  **Privacy Amplification:** Even if Eve didn't know the whole key, she might know *some* of it. They use a hash function to "shrink" the key. If Eve knew 10% of the bits, they compress the key so her knowledge of the final result drops to near zero.

The final secure key rate is given by:
$$ R \approx 1 - H_1(e) - H_2(e) $$
Where $H(e)$ is the cost of correcting errors and amplifying privacy.

### 8. Privacy Amplification (Simplified)
To wipe out Eve's partial knowledge, we shrink the key using a hash function. A simple hash is to XOR random subsets of the key.

**Task:** Create a new 1-bit final key by XORing all the bits in the raw key together.

In [12]:
# Simple Privacy Amplification: XOR all bits to get 1 final secure bit
final_secure_bit_alice = 0
for bit in alice_key_part:
    final_secure_bit_alice ^= bit
    
final_secure_bit_bob = 0
for bit in raw_key_attacked:
    final_secure_bit_bob ^= bit

print(f"Alice's Final Key: {final_secure_bit_alice}")
print(f"Bob's Final Key:   {final_secure_bit_bob}")

Alice's Final Key: 0
Bob's Final Key:   1
